In [9]:
import cv2
import numpy as np
import os

In [10]:

def align_images(input_directory, output_directory, reference_image_path):
    # Ensure the output directory exists
    os.makedirs(output_directory, exist_ok=True)

    # Read the reference image
    ref_image = cv2.imread(reference_image_path)
    if ref_image is None:
        print("Error: Reference image not found.")
        return

    # Convert reference image to grayscale
    ref_gray = cv2.cvtColor(ref_image, cv2.COLOR_BGR2GRAY)
    
    # Initialize ORB detector
    orb = cv2.ORB_create()

    # Find keypoints and descriptors of the reference image
    ref_keypoints, ref_descriptors = orb.detectAndCompute(ref_gray, None)

    # Iterate through each image in the input directory
    for image_name in os.listdir(input_directory):
        image_path = os.path.join(input_directory, image_name)

        # Read the image
        image = cv2.imread(image_path)
        if image is None:
            continue

        # Convert image to grayscale
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Find keypoints and descriptors of the current image
        keypoints, descriptors = orb.detectAndCompute(gray_image, None)

        # Create a BFMatcher object
        matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

        # Match descriptors
        matches = matcher.match(ref_descriptors, descriptors)
        
        # Sort matches based on their distance
        matches = sorted(matches, key=lambda x: x.distance)

        # Draw matches (optional for visualization)
        matched_image = cv2.drawMatches(ref_image, ref_keypoints, image, keypoints, matches[:10], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

        # Get the corresponding points
        src_pts = np.float32([ref_keypoints[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([keypoints[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

        # Find the homography matrix
        M, _ = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC)

        # Align the image
        aligned_image = cv2.warpPerspective(image, M, (ref_image.shape[1], ref_image.shape[0]))

        # Save the aligned image
        output_path = os.path.join(output_directory, image_name)
        cv2.imwrite(output_path, aligned_image)

        # Optional: Save matched image for debugging
        matched_output_path = os.path.join(output_directory, f"matched_{image_name}")
        cv2.imwrite(matched_output_path, matched_image)

In [11]:
input_directory = 'Data'  # Replace with your input directory
output_directory = 'Output'  # Replace with your output directory
reference_image_path = 'Reference.jpg'  # Replace with your reference image path
align_images(input_directory, output_directory, reference_image_path)